# Chaining and Retrieval

In [44]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("sop.pdf")
docs = loader.load()

In [45]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(docs)

In [46]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents, embeddings)

In [47]:
from langchain_community.llms import Ollama
# load llama2 model
llm=Ollama(model="llama2")

In [48]:
# Design prompt template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the context provided below.
This is a standard operating procedure document. Give precise, to-the-point answers.
Give the answer within 100 words (more if required).
<context>
{context}
</context>
Question: {input}""")

In [49]:
# chain introduction
# Create document chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [50]:
"""Retrivers: An interface which connects to the vector db and returns documents given an unstructured query
It does not need to be able to store the documents, just return them."""

retriver = db.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B471FDC090>, search_kwargs={})

In [51]:
# Retriever Chain
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriver, document_chain)

In [53]:
retrieval_chain.invoke({"input" : "What is this SOP about?"})['answer']

'This Standard Operating Procedure (SOP) document provides guidelines for the development, implementation, and review of Standard Operating Procedures (SOPs) in commercial aviation. It emphasizes the importance of collaboration among various stakeholders, including flightcrews, managers, check airmen, and instructors, to ensure that SOPs are effective, efficient, and safe. The document also highlights the need for ongoing review and revision of SOPs to adapt to changing circumstances and improve safety culture.'